figure 4(a) drawing code

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap

# ========== Load Data ==========
df = pd.read_csv("/Users/yujiezhu/Desktop/Postdoc_research/2_transient_progress/2_code/0_Figure1_analytical/0_data2/1_STO/pto_phase_results_STO_50K.csv")

# Extract required columns
epsilon_app = df['epsilonApp'].values
theta = df['theta'].values
P1 = df['P1'].values
P2 = df['P2'].values

# Sort data by epsilonApp and theta
df_sorted = df.sort_values(by=['epsilonApp', 'theta']).reset_index(drop=True)

custom_xticks = [0.00, 0.005, 0.01, 0.015, 0.0198]  
custom_yticks = [0, np.pi / 8, np.pi / 4, 3*np.pi / 8, np.pi / 2]  
custom_ytick_labels = ["0", r"$\frac{\pi}{8}$", r"$\frac{\pi}{4}$", r"$\frac{3 \pi}{8}$", r"$\frac{\pi}{2}$"]

# Extract sorted values
epsilon_app = df_sorted['epsilonApp'].values
theta = df_sorted['theta'].values
P1 = df_sorted['P1'].values
P2 = df_sorted['P2'].values

# Compute the angle with the x1-axis
angle = np.arctan2(P2, P1)  # Angle in radians

# Apply Constraints:
angle[(theta < np.pi / 4) & (angle > np.pi / 4) & (epsilon_app>0.003)] = 0
angle[(theta > np.pi / 4) & (angle < np.pi / 4) & (epsilon_app>0.003)] = np.pi / 2

# Condition: If P1 and P2 are both below 1e-3, mark as NaN for gray regions
mask = (np.abs(P1) < 1e-3) & (np.abs(P2) < 1e-3)
angle[mask] = np.nan  

# Get unique values for reshaping
epsilon_app_unique = np.unique(epsilon_app)
theta_unique = np.unique(theta)

# Reshape data into 2D grids
epsilon_app_grid = epsilon_app.reshape(len(epsilon_app_unique), len(theta_unique))
theta_grid = theta.reshape(len(epsilon_app_unique), len(theta_unique))
angle_grid = angle.reshape(len(epsilon_app_unique), len(theta_unique))

# ========== Create Plot ==========
plt.figure(figsize=(8, 6))

# Create colormap: Use 'bwr' for normal values, gray for NaN values
orig_cmap = cm.get_cmap('RdYlBu')
trimmed_cmap = LinearSegmentedColormap.from_list(
    "trimmed_RdYlBu", orig_cmap(np.linspace(0.15, 0.85, 256))
)

trimmed_cmap.set_bad(color='gray')

# Heatmap with user-defined color range (0 to π/2)
mesh = plt.pcolormesh(epsilon_app_unique, theta_unique, angle_grid.T, shading='auto', cmap=trimmed_cmap, \
    alpha= 1, norm=Normalize(vmin=0, vmax=np.pi/2))

# Colorbar settings
cbar = plt.colorbar(mesh, label=r'Angle with x1-axis ($\theta$)')
cbar.set_ticks([0, np.pi/4, np.pi/2])
cbar.set_ticklabels([r'$0$', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$'])

# Set custom ticks for theta (y-axis)
plt.yticks(custom_yticks, custom_ytick_labels, fontsize=12)
# Set custom ticks for epsilonApp (x-axis)
plt.xticks(custom_xticks, fontsize=12)

# Axis labels and title
plt.title('Angle Between (P1, P2) and x1-Axis (with Constraints)', fontsize=14)
plt.xlabel('epsilonApp', fontsize=12)
plt.ylabel('theta (radians)', fontsize=12)

plt.tight_layout()
plt.show()

# ========== Save Processed Data ==========
df_processed = pd.DataFrame({
    'epsilonApp': epsilon_app_grid.flatten(),
    'theta': theta_grid.flatten(),
    'Angle': angle_grid.flatten()
})


print("Processed results saved to 'pto_phase_results_angle_filtered_constraints.csv'")

figure 1 analytical calculation code

In [ ]:
import pandas as pd
from joblib import Parallel, delayed
import numpy as np

# STO uniaxially strained membrane 300K 2% applied strain

epsilon_0 = 8.854187817e-12
c = 299792458
gamma1 = 2e-7

mu = 22e-18
d = 10e-9
gamma = gamma1 + d / (2 * epsilon_0 * c)
a_val = -(gamma / (2 * mu))

# T = 300 K
Kxx = 4.80309e8
Kyy = 8.18106e8
bxx_val = np.sqrt(4 * mu * Kxx - gamma**2) / (2 * mu)
byy_val = np.sqrt(4 * mu * Kyy - gamma**2) / (2 * mu)

# J=1e11
EE = 18795.674 * 700
tau0 = 0.3e-12

t0 = 5 * tau0
omega = 2 * np.pi * 1e12


CCxxx = 5.34947e9
CCxyy = 2.021e9
CCyxy = 2.021e9


def Fx(z):
    return EE * np.exp(-((z - t0)**2) / (2*tau0**2)) * np.cos(omega * (z - t0) + 0*np.pi/2)

# 使用 trapz 积分并缓存结果
def trapz_integrate(func, a, b, num_points=1000):
    x = np.linspace(a, b, num_points)
    y = func(x)
    return np.trapz(y, x)

# 优化后的 Px 和 Py 计算
def P_linear(t, b_val, func):
    integral_1 = trapz_integrate(lambda z: np.exp(-a_val * z) * np.sin(b_val * z) * func(z), 0, t)
    integral_2 = trapz_integrate(lambda z: np.exp(-a_val * z) * np.cos(b_val * z) * func(z), 0, t)

    part1 = np.exp(a_val * t) * np.cos(b_val * t) * (-1 / (mu * b_val) * integral_1)
    part2 = np.exp(a_val * t) * np.sin(b_val * t) * ( 1 / (mu * b_val) * integral_2)

    return part1 + part2

def Px_linear(t): return P_linear(t, bxx_val, Fx)
def Px_nonlinear1(t): return P_linear(t, bxx_val, lambda z: -CCxxx * Px_linear(z)**2)
def Px_nonlinear2(t): return P_linear(t, bxx_val, lambda z: -2*1 * Px_linear(z)*Px_nonlinear1(z))

# Px_tot 和 Py_tot 计算优化
def Px_tot(t):
    Px_lin = Px_linear(t)
    Px_nonlin1 = Px_nonlinear1(t)
    #Px_nonlin2 = Px_nonlinear2(t)
    return Px_lin + Px_nonlin1 


def E_linear(t, b_val, func):
    integral_1 = trapz_integrate(lambda z: np.exp(-a_val * z) * np.sin(b_val * z) * func(z), 0, t)
    integral_2 = trapz_integrate(lambda z: np.exp(-a_val * z) * np.cos(b_val * z) * func(z), 0, t)

    part1 = (-a_val*np.cos(b_val*t) + b_val*np.sin(b_val*t)) * integral_1
    part2 = ( a_val*np.sin(b_val*t) + b_val*np.cos(b_val*t)) * integral_2

    return - d * np.exp(a_val * t) /(2* b_val*c*mu*epsilon_0) * (part1 + part2)

def Ex_linear(t): return E_linear(t, bxx_val, Fx)
def Ex_nonlinear1(t): return E_linear(t, bxx_val, lambda z: -CCxxx * Px_linear(z)**2)
def Ex_nonlinear2(t): return E_linear(t, bxx_val, lambda z: -2*CCxxx * Px_linear(z)*Px_nonlinear1(z))

# Ex_rad 和 Ey_rad 计算优化
def Ex_rad(t):
    Ex_lin = Ex_linear(t)
    Ex_nonlin1 =  Ex_nonlinear1(t)
    #Ex_nonlin2 =  Ex_nonlinear2(t)
    return Ex_lin + Ex_nonlin1 


# Ex_tot 和 Ey_tot 计算优化
def Ex_tot(t):
    Ex_lin = Ex_linear(t)
    Ex_back = Fx(t)
    Ex_nonlin1 =  Ex_nonlinear1(t)
    Ex_nonlin2 =  Ex_nonlinear2(t)
    return Ex_back + Ex_lin + Ex_nonlin1 + 0*Ex_nonlin2


# 并行计算 Px_tot 和 Py_tot
def calculate_parallel(func, t_values):
    return Parallel(n_jobs=2)(delayed(func)(t) for t in t_values)

def fft_analysis(time_data, signal_data):
    # 计算采样间隔
    sampling_interval = np.mean(np.diff(time_data))
    # 计算采样频率
    sampling_rate = 1.0 / sampling_interval

    # 执行 FFT
    fft_result = np.fft.fft(signal_data)
    n = len(fft_result)
    # 计算频率（只取一半，因为 FFT 结果是对称的）
    freq = np.fft.fftfreq(n, d=sampling_interval)[:n // 2]

    # 计算幅度（取绝对值）
    amplitude = 2.0 / n * np.abs(fft_result[:n // 2])
    # 获取实部和虚部
    real_part = np.real(fft_result[:n // 2])
    imag_part = np.imag(fft_result[:n // 2])

    return freq, amplitude, real_part, imag_part




# this is for the analytical solution

# 时间范围
time_ps = np.linspace(0, 200, 20001)
time_sec = time_ps * 1e-12


time_ps2 = np.linspace(0, 200, 20001)
time_sec2 = time_ps2 * 1e-12
EEx = Fx(time_sec2)



# 计算 Px_tot 和 Py_tot 并缓存
Px_values = np.array(calculate_parallel(Px_tot, time_sec))
Px_linear_values = np.array(calculate_parallel(Px_linear, time_sec))
Ex_values = np.array(calculate_parallel(Ex_tot, time_sec))
Ex_rad_values = np.array(calculate_parallel(Ex_rad, time_sec))


df = pd.DataFrame({
    "time": time_sec,
    "Px": Px_values,
    "E_inc": EEx,
    "E_tot": Ex_values,
    "E_rad": Ex_rad_values
})
# 保存到 CSV
df.to_csv("0_extract_data.csv", index=False)



freq, amplitude1, real_part1, imag_part1 = fft_analysis(time_sec, EEx)
freq = freq
mask = (freq >= 0) & (freq <= 7e12)
freq_selected = freq[mask]
amplitude1_selected = amplitude1[mask]
real_part1_selected = real_part1[mask]
imag_part1_selected = imag_part1[mask]



freq, amplitude2, real_part2, imag_part2 = fft_analysis(time_sec, Px_values)
freq = freq
mask = (freq >= 0) & (freq <= 7e12)
freq_selected = freq[mask]
amplitude2_selected = amplitude2[mask]
real_part2_selected = real_part2[mask]
imag_part2_selected = imag_part2[mask]



freq, amplitude3, real_part3, imag_part3 = fft_analysis(time_sec, Ex_rad_values)
freq = freq
mask = (freq >= 0) & (freq <= 7e12)
freq_selected = freq[mask]
amplitude3_selected = amplitude3[mask]
real_part3_selected = real_part3[mask]
imag_part3_selected = imag_part3[mask]




freq, amplitude4, real_part4, imag_part4 = fft_analysis(time_sec, Ex_values)
freq = freq
mask = (freq >= 0) & (freq <= 7e12)
freq_selected = freq[mask]
amplitude4_selected = amplitude4[mask]
real_part4_selected = real_part4[mask]
imag_part4_selected = imag_part4[mask]



df = pd.DataFrame({
    "Frequency (Hz)": freq_selected,
    "Amplitude_Px": amplitude2_selected/np.max(amplitude2_selected),
    "Amplitude_Ex_inc": amplitude1_selected/np.max(amplitude1_selected),
    "Amplitude_Ex_tot": amplitude4_selected/np.max(amplitude4_selected),
    "Amplitude_Ex_rad": amplitude3_selected/np.max(amplitude3_selected)
})
# 保存到 CSV
df.to_csv("0_fft_combined.csv", index=False)


import matplotlib.pyplot as plt

# --------- 图 1: Px vs. Time ---------
plt.figure(figsize=(10, 4))
plt.plot(time_sec, EEx, color='blue', linewidth=2)
plt.xlim([0, 10e-11])
plt.xlabel('Time (s)')
plt.ylabel('Px (total)')
plt.title('Polarization Px vs. Time')
plt.grid(True)
plt.tight_layout()
plt.show()

# --------- 图 1: Px vs. Time ---------
plt.figure(figsize=(10, 4))
plt.plot(freq_selected, amplitude1_selected, color='blue', linewidth=2)
plt.xlabel('f')
plt.ylabel('amp')
plt.title('FFT')
plt.grid(True)
plt.tight_layout()
plt.show()

# --------- 图 1: Px vs. Time ---------
plt.figure(figsize=(10, 4))
plt.plot(time_sec, Px_values, color='blue', linewidth=2)
plt.xlim([0, 10e-11])
plt.xlabel('Time (s)')
plt.ylabel('Px (total)')
plt.title('Polarization Px vs. Time')
plt.grid(True)
plt.tight_layout()
plt.show()

# --------- 图 1: Px vs. Time ---------
plt.figure(figsize=(10, 4))
plt.plot(freq_selected, np.log(amplitude2_selected), color='blue', linewidth=2)
plt.xlabel('f')
plt.ylabel('amp')
plt.title('FFT')
plt.grid(True)
plt.tight_layout()
plt.show()

# --------- 图 2: Ex vs. Time ---------
plt.figure(figsize=(10, 4))
plt.plot(time_sec, Ex_rad_values, color='red', linewidth=2)
plt.xlim([0, 10e-11])
plt.xlabel('Time (s)')
plt.ylabel('Ex (total)')
plt.title('Electric Field Ex vs. Time')
plt.grid(True)
plt.tight_layout()
plt.show()

# --------- 图 1: Px vs. Time ---------
plt.figure(figsize=(10, 4))
plt.plot(freq_selected, np.log(amplitude3_selected), color='blue', linewidth=2)
plt.xlabel('f')
plt.ylabel('amp')
plt.title('FFT')
plt.grid(True)
plt.tight_layout()
plt.show()

figure 3(g) calculation code

In [ ]:
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

epsilon_app_value0 = 0.005
theta_value0 = 0*np.pi




epsilon_0 = 8.854187817e-12
c = 299792458
gamma1 = 2e-7

mu = 1.081e-18
d = 10e-9
gamma = gamma1 + d / (2 * epsilon_0 * c)
a_val = -(gamma / (2 * mu))

KKxx = 1.74978e10
KKyy = 3.23702e8
f0_z_value = np.sqrt(KKxx/mu)/(2*np.pi)/ 1e12

bxx_val = np.sqrt(4 * mu * KKxx - gamma**2) / (2 * mu)
byy_val = np.sqrt(4 * mu * KKyy - gamma**2) / (2 * mu)

# J=1e11
EE = 18795.674 * 1
tau0 = 1e-12

t0 = 3 * tau0
omega = 2 * np.pi * f0_z_value *1e12



def Fx(z):
    return -EE * np.exp(-((z - t0)**2) / (2*tau0**2)) * np.sin(omega * (z - t0))

def Fy(z):
    return -EE * np.exp(-((z - t0)**2) / (2*tau0**2)) * np.cos(omega * (z - t0))







# 使用 trapz 积分并缓存结果
def trapz_integrate(func, a, b, num_points=1000):
    x = np.linspace(a, b, num_points)
    y = func(x)
    return np.trapz(y, x)





def P_linear(t, b_val, func):
    integral_1 = trapz_integrate(lambda z: np.exp(-a_val * z) * np.sin(b_val * z) * func(z), 0, t)
    integral_2 = trapz_integrate(lambda z: np.exp(-a_val * z) * np.cos(b_val * z) * func(z), 0, t)

    part1 = np.exp(a_val * t) * np.cos(b_val * t) * (-1 / (mu * b_val) * integral_1)
    part2 = np.exp(a_val * t) * np.sin(b_val * t) * ( 1 / (mu * b_val) * integral_2)

    return part1 + part2

def Px_linear(t): return P_linear(t, bxx_val, Fx)
def Py_linear(t): return P_linear(t, byy_val, Fy)





def Px_tot(t):
    Px_lin = Px_linear(t)
    return Px_lin

def Py_tot(t):
    return Py_linear(t)


def E_linear(t, b_val, func):
    integral_1 = trapz_integrate(lambda z: np.exp(-a_val * z) * np.sin(b_val * z) * func(z), 0, t)
    integral_2 = trapz_integrate(lambda z: np.exp(-a_val * z) * np.cos(b_val * z) * func(z), 0, t)

    part1 = (-a_val*np.cos(b_val*t) + b_val*np.sin(b_val*t)) * integral_1
    part2 = ( a_val*np.sin(b_val*t) + b_val*np.cos(b_val*t)) * integral_2

    return - d * np.exp(a_val * t) /(2* b_val*c*mu*epsilon_0) * (part1 + part2)

def Ex_linear(t): return E_linear(t, bxx_val, Fx)
def Ey_linear(t): return E_linear(t, byy_val, Fy)






def Ex_rad(t):
    Ex_lin = Ex_linear(t)
    return Ex_lin 

def Ey_rad(t):
    return Ey_linear(t)


def Ex_tot(t):
    Ex_lin = Ex_linear(t)
    Ex_back = Fx(t)
    return Ex_back + Ex_lin

def Ey_tot(t):
    Ey_back = Fy(t)
    return Ey_back + Ey_rad(t)



def calculate_parallel(func, t_values):
    return Parallel(n_jobs=-1)(delayed(func)(t) for t in t_values)





def plot_colored_trajectory_E(Px_values, Py_values, name, alpha=0.1, tick_size=12, xlim=None, ylim=None, tick_length=6, tick_width=2, spine_width=2):
    rotation_direction = np.zeros(len(Px_values) - 1)
    for i in range(1, len(Px_values) - 1):
        dx1 = Px_values[i] - Px_values[i - 1]
        dy1 = Py_values[i] - Py_values[i - 1]
        dx2 = Px_values[i + 1] - Px_values[i]
        dy2 = Py_values[i + 1] - Py_values[i]
        
        
        cross_product = dx1 * dy2 - dy1 * dx2
        rotation_direction[i] = cross_product
    

    plt.figure(figsize=(8, 8))
    for i in range(len(rotation_direction)):
        color = 'blue' if rotation_direction[i] > -0.01 else 'red' 
        plt.plot(Px_values[i:i + 2], Py_values[i:i + 2], color=color, alpha=alpha)
    

    plt.xlabel("Px_tot", fontsize=tick_size)
    plt.ylabel("Py_tot", fontsize=tick_size)
    #plt.title("Trajectory: Left (Blue) and Right (Red) Rotation", fontsize=tick_size)


    plt.xticks(fontsize=tick_size)
    plt.yticks(fontsize=tick_size)


    plt.tick_params(axis='both', which='both', length=tick_length, width=tick_width)


    ax = plt.gca()
    for spine in ax.spines.values():
        spine.set_linewidth(spine_width)


    if xlim is not None:
        plt.xlim(xlim)
    if ylim is not None:
        plt.ylim(ylim)


    plt.savefig(name)
    plt.show()





def plot_3d_trajectory_with_rotation_horizontal(time_steps, Px_values, Py_values, name, time_to_ex_ratio=2, elev=30, azim=120):

    time_steps = np.array(time_steps)
    Px_values = np.array(Px_values)
    Py_values = np.array(Py_values)


    rotation_direction = np.zeros(len(time_steps) - 1)
    for i in range(1, len(time_steps) - 1):
        dx1 = Px_values[i] - Px_values[i - 1]
        dy1 = Py_values[i] - Py_values[i - 1]
        dx2 = Px_values[i + 1] - Px_values[i]
        dy2 = Py_values[i + 1] - Py_values[i]
        

        cross_product = dx1 * dy2 - dy1 * dx2
        rotation_direction[i] = cross_product


    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')


    for i in range(len(rotation_direction)):
        color = 'blue' if rotation_direction[i] >= -0.01 else 'red'  
        ax.plot(time_steps[i:i + 2], Px_values[i:i + 2], Py_values[i:i + 2], alpha=0.6, color=color, linewidth=1)  


    fig.patch.set_alpha(0)  
    ax.patch.set_alpha(0)   

    ax.set_xlim((0,6*tau0))


    # 1.21 THz
    
    ax.set_ylim((-20e3,20e3))
    ax.set_zlim((-20e3,20e3))

    ax.set_xticks([0, 2, 4, 6, 8])
    ax.set_yticks([-20e3, 0, 20e3])
    ax.set_zticks([-20e3, 0, 20e3])
    


    #ax.set_xlabel("Time Step", fontsize=14)
    ax.set_ylabel("Px_tot", fontsize=14)
    ax.set_zlabel("Py_tot", fontsize=14)


    ax.set_box_aspect([time_to_ex_ratio, 1, 1])  


    #ax.tick_params(axis='both', which='major', labelsize=12, color='black', labelcolor='white', length=10, width=2)


    ax.xaxis.label.set_color((1, 1, 1, 0))  
    ax.tick_params(axis='x', which='both', labelsize=12, color='black', labelcolor=(1, 1, 1, 0), length=500, width=6) 

    ax.yaxis.label.set_color((1, 1, 1, 0))  

    ax.yaxis.set_tick_params(colors=(1.0, 1.0, 1.0, 0.0))  
    ax.yaxis.line.set_color((1.0, 1.0, 1.0, 0.0))  
    
    ax.zaxis.label.set_color((1, 1, 1, 0)) 

    ax.zaxis.set_tick_params(colors=(1.0, 1.0, 1.0, 0.0))  
    ax.zaxis.line.set_color((1.0, 1.0, 1.0, 0.0)) 




    ax.view_init(elev=elev, azim=azim)


    ax.grid(False)


    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)


    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))  
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))  
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0)) 


    plt.savefig(name)
    plt.show()






# this is for the analytical solution


time_ps = np.linspace(0, 8, 10001)
time_sec = time_ps * 1e-12


time_ps2 = np.linspace(0, 8, 10001)
time_sec2 = time_ps2 * 1e-12
EEx = Fx(time_sec2)
EEy = Fy(time_sec2)



Px_values = calculate_parallel(Px_tot, time_sec)
Py_values = calculate_parallel(Py_tot, time_sec)

Ex_values = calculate_parallel(Ex_tot, time_sec)
Ey_values = calculate_parallel(Ey_tot, time_sec)

Ex_rad_values = calculate_parallel(Ex_rad, time_sec)
Ey_rad_values = calculate_parallel(Ey_rad, time_sec)



plot_colored_trajectory_E(Ex_values, Ey_values, '0_trajectory_plot_E.png', alpha=0.5, tick_size=14, xlim=(-20000, 20000), ylim=(-20000, 20000),\
                           tick_length=8, tick_width=1.5, spine_width=1.5)



plot_3d_trajectory_with_rotation_horizontal(time_ps, Ex_values, Ey_values, '0_trajectory_plot_E_3d.png', time_to_ex_ratio=2, \
                                            elev=30, azim=-75)


plot_3d_trajectory_with_rotation_horizontal(time_ps, EEx, EEy, \
                                            '0_trajectory_plot_Einc_3d.png', time_to_ex_ratio=2, \
                                            elev=30, azim=-75)